## Rocket launches, a best value anaylsis.
<p>Launching payloads, such as satellites or space station modules, into space is <strong>expensive</strong> and requires careful planning (see <a href="https://www.businessinsider.com/spacex-rocket-cargo-price-by-weight-2016-6?r=US&IR=T">this great article on the topic</a>). Finding and investing in regions that have a developed space industry can reduce costs and improve the pace of space travel and exploration.</p>
<p>The European Space Agency (ESA) has decided to invest in countries that are home to the most efficient companies at sending payloads into space. In particular, they want to find countries with the lowest <strong>average price per kilo</strong> (across all companies for each country) that can launch <strong>rockets</strong> into <strong>Low Earth Orbit (LEO)</strong> according to each of their three classes (light, medium, and heavy rockets). </p>
<p>However, there are some additional contraints that the ESA has put in place for the selection process:</p>
<ul>
<li>Companies included in each country's average price must have a Quality Assurance (QA) rating <strong>higher</strong> than 2. QA is a rating based on previous launches. This constraint establishes a minimum level of performance for the selection process.</li>
<li>Countries with a <strong>launch cost</strong> under \$10,000,000 in total for <strong>all rocket launches across the three classes</strong> should be excluded. This constraint ensures that any selected countries will be able to manage the large volume of launches required.</li>
</ul>
<p>To help you in your task, the ESA has made available to you three datasets: single-owner companies (<code>datasets/SO-space.csv</code>) and joint-ventures (<code>datasets/JV-space.csv</code>), both of which contain launch data on companies that offer space services. These are paired with the company info dataset (<code>datasets/company_info.csv</code>) which contains the company names, launch technology (rockets, balloons, planes, other), and their location details.</p>
<p>The launch classes are defined as follows based on their payload in Kilograms:</p>
<ul>
<li>Light &lt;= 1,000</li>
<li>1,000 &lt; Medium &lt;= 10,000</li>
<li>10,000 &lt; Heavy </li>
</ul>
<p><em><strong>Please note that 1000 kg = 1 metric ton.</strong></em></p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/SO-space.csv - Space launch data for private single owner companies</b>
    </div>
    <div>Source: <a href="https://spacefund.com">SpaceFund</a></div>
<ul>
    <li><b>Company ID: </b>The company ID.</li>
    <li><b>QA: </b>The quality assurance rating as given by an appointed third party agency.</li>
    <li><b>Payload (kg): </b>The weight in kilograms for the largest spacecraft launched by the company.</li>
    <li><b>Launch Cost (\$M): </b>The launch cost for the largest spacecraft, in millions of USD.</li>
    <li><b>Price (\$/Kg): </b>The price paid to hire the company services by a new client, in USD per kilogram.</li>
    <li><b>Orbit Altitude: </b>The earth orbit achieved by the company's spacecraft.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/JV-space.csv - Space launch data for private joint-venture companies</b>
    </div>
    <div>Source: <a href="https://spacefund.com">SpaceFund</a></div>
<ul>
    <li><b>Company ID: </b>The company ID.</li>
    <li><b>QA: </b>The quality assurance rating as given by an appointed third party agency.</li>
    <li><b>Payload (tons): </b>The weight in tons for the largest spacecraft launched by the company.</li>
    <li><b>Launch Cost: </b>The launch cost for the largest spacecraft, in USD.</li>
    <li><b>Price (\$/ton): </b>The price paid to hire the company services by a new client, in USD per ton.</li>
    <li><b>Orbit Altitude: </b>The earth orbit achieved by the company's spacecraft.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 25px 25px 15px 25px; margin-top: 17px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/company_info.csv - Company name, launch technology, and geographical data</b>
    </div>
    <div>Source: <a href="https://spacefund.com">SpaceFund</a></div>
<ul>
    <li><b>ID: </b>The company ID.</li>
    <li><b>Company: </b>The company name.</li>
    <li><b>Tech Type: </b>The spacecraft type. Rocket, Spaceplace, Plane, Balloon, or Other.</li>
    <li><b>Country: </b>The country location for the company's launchpad.</li>
    <li><b>HQ Location: </b>The main office location fot the company.</li>
</ul>
    </div>

In [66]:
import pandas as pd
company_info = pd.read_csv('datasets/company_info.csv')
jv = pd.read_csv('datasets/JV-space.csv')
sospace = pd.read_csv('datasets/SO-space.csv')
jv['Payload (kg)'] = round(jv['Payload (tons)'] * 907.185, 3)
jv['Price ($/kg)'] = round(jv['Price ($/ton)'] * 0.00110231131, 3)
jv['Launch Cost ($M)'] =jv['Launch Cost'].str.replace(',','').astype('int') / 1000000
to_drop = ['Price ($/ton)','Launch Cost', 'Payload (tons)']
jv = jv.drop(to_drop, axis = 1)
sospace = sospace[['Company ID', 'QA', 'Orbit Altitude','Payload (kg)','Price ($/kg)', 'Launch Cost ($M)']]

In [67]:
jvinfo = pd.concat([jv, sospace])
my_df = company_info.merge(jvinfo, left_on = 'ID', right_on = 'Company ID')
my_df2 = my_df[(my_df['Tech Type'] == 'Rocket') & (my_df['Orbit Altitude'] == 'LEO')]

In [68]:
dropcols = ['Company', 'HQ Location', 'QA', 'Company ID']
my_df2 = my_df2.drop(dropcols, axis = 1)

In [69]:
my_df2 = my_df2.dropna()
my_df2['Price ($/kg)'] = my_df2['Price ($/kg)'].str.replace(',',"").astype('int')
my_df2
#my_df2['Average Price'] = my_df2.groupby('Country')['Price ($/kg)'].mean()

ValueError: cannot convert float NaN to integer

In [0]:
%%nose
import pandas as pd
from pandas.api.types import is_numeric_dtype
convert_index = lambda x: [re.match('(\d{4})', date).group(0) for date in x.index.values.astype(str)]

l_class = ["Light", "Medium", "Heavy"]
a_price = [15560.0, 9663.5, 3711.5]
country = ["Germany", "USA", "USA"]
test_solution = pd.DataFrame({"Launch Class": l_class, "Average Price": a_price, "Country": country})

test_columns = ['Launch Class', 'Average Price', 'Country']
def test_project():
    # Check whether the answer has been saved and is a DataFrame
    assert 'launch_cost' in globals() and type(launch_cost) == pd.core.frame.DataFrame, \
    "Have you assigned your answer to a DataFrame named `launch_cost`?"
    # Check whether they have the right columns in their DataFrame
    assert launch_cost.columns.isin(test_columns).all(), \
    "Your DataFrame is missing the required columns!"
    # Check whether the values (converted to an integer) contain in the only column are correct
    # and check whether the index is identical
    for column in test_columns:
        # For the price column (the numeric one), convert to an integer and compare
        if is_numeric_dtype(test_solution[column]):
            assert (test_solution[column].astype(int).values == launch_cost[column].astype(int).values).all(), \
            "Your submitted DataFrame does not contain the correct values!"
        # For the non-numeric columns, just compare equality (there shouldn't have been string operations)
        else:
            assert test_solution[column].str.lower().equals(launch_cost[column].str.lower()), \
            "Your submitted DataFrame does not contain the correct values!"